# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.dropna(inplace = True) 
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,college,64.8569,-147.8028,5.00,65.0,20,1.61,US,1.613183e+09
1,boende,-0.2167,20.8667,75.00,70.0,100,1.30,CD,1.613183e+09
2,martapura,-3.4167,114.8500,82.40,83.0,40,4.61,ID,1.613183e+09
3,east london,-33.0153,27.9116,74.79,80.0,12,11.63,ZA,1.613183e+09
4,clearwater,27.9659,-82.8001,72.00,94.0,100,4.32,US,1.613183e+09
...,...,...,...,...,...,...,...,...,...
532,santander jimenez,24.2167,-98.4667,50.56,61.0,100,10.18,MX,1.613184e+09
533,eldorado,24.3234,-107.3636,68.00,68.0,75,3.58,MX,1.613184e+09
534,huarmey,-10.0681,-78.1522,73.22,68.0,98,6.67,PE,1.613184e+09
535,manica,-18.9367,32.8760,71.60,94.0,75,6.91,MZ,1.613184e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure API Gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [4]:
# Add Heatmap layer to map.
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:
A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Note: Feel free to adjust your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number; e.g., 10.


In [5]:
filtered_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & (weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)].dropna()

filtered_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,hermanus,-34.4187,19.2345,71.01,81.0,0,3.00,ZA,1.613183e+09
73,busselton,-33.6500,115.3333,75.00,58.0,0,1.99,AU,1.613183e+09
75,kaeo,-35.1000,173.7833,75.00,61.0,0,5.99,NZ,1.613183e+09
77,esperance,-33.8667,121.9000,73.40,46.0,0,5.75,AU,1.613183e+09
134,gunjur,13.2019,-16.7339,71.60,49.0,0,5.75,GM,1.613183e+09
147,tecoanapa,16.5167,-98.7500,75.34,83.0,0,6.96,MX,1.613184e+09
244,batie,9.8833,-2.9167,74.73,19.0,0,3.91,BF,1.613184e+09
346,ahipara,-35.1667,173.1667,71.64,57.0,0,2.33,NZ,1.613184e+09
383,terre-de-bas,15.8501,-61.6442,73.40,78.0,0,6.91,GP,1.613184e+09
390,nicoya,10.1483,-85.4520,77.00,79.0,0,5.75,CR,1.613184e+09


Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = filtered_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
41,hermanus,ZA,-34.4187,19.2345,
73,busselton,AU,-33.6500,115.3333,
75,kaeo,NZ,-35.1000,173.7833,
77,esperance,AU,-33.8667,121.9000,
134,gunjur,GM,13.2019,-16.7339,
147,tecoanapa,MX,16.5167,-98.7500,
244,batie,BF,9.8833,-2.9167,
346,ahipara,NZ,-35.1667,173.1667,
383,terre-de-bas,GP,15.8501,-61.6442,
390,nicoya,CR,10.1483,-85.4520,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel", # Type of search
          "keyword" : "hotel", # Keyword used for searching
          "radius" : 5000, # Circle Radius containing hotel 
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # save info in variables 
    latitude = row["Lat"]
    longitude = row["Lng"]
    name = row["City"]
    
    print(f"Processing {name}.")
    
    # add location to params 
    params["location"] = f"{latitude},{longitude}"

    # API request and get response in json format
    response = requests.get(base_url, params=params).json()
    
    # results is the main key in json response
    results = response['results']
    
    # Update the hotel name to hotel_df['Hotal Name'] column 
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if search failed
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to avoid exceed api requests allowed for free
    time.sleep(1)

print("End reached")

hotel_df.dropna(inplace=True, how='all')
hotel_df

Processing hermanus.
------------
Processing busselton.
------------
Processing kaeo.
------------
Processing esperance.
------------
Processing gunjur.
------------
Processing tecoanapa.
------------
Processing batie.
Missing field/result... skipping.
------------
Processing ahipara.
------------
Processing terre-de-bas.
------------
Processing nicoya.
------------
Processing umm lajj.
------------
Processing veraval.
------------
Processing ziniare.
------------
Processing keti bandar.
Missing field/result... skipping.
------------
Processing tabou.
------------
Processing dargaville.
------------
Processing lakes entrance.
------------
End reached


,City,Country,Lat,Lng,Hotel Name
41,hermanus,ZA,-34.4187,19.2345,Misty Waves Boutique Hotel
73,busselton,AU,-33.6500,115.3333,The Sebel Busselton
75,kaeo,NZ,-35.1000,173.7833,Pacific Harbour Lodge
77,esperance,AU,-33.8667,121.9000,The Jetty Resort
134,gunjur,GM,13.2019,-16.7339,Dalaba Eco-Lodge
147,tecoanapa,MX,16.5167,-98.7500,Hotel Garapacho
244,batie,BF,9.8833,-2.9167,
346,ahipara,NZ,-35.1667,173.1667,The Huts
383,terre-de-bas,GP,15.8501,-61.6442,Hôtel Bois Joli
390,nicoya,CR,10.1483,-85.4520,Hotel Doña Marta


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer and info box content ontop of heat map
# https://jupyter-gmaps.readthedocs.io/en/latest/api.html

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))